# <strong>Pathname Attacks</strong>

Pathname attacks, also known as directory traversal attacks, are exploits that allow an attacker to view files that are not stored within the web server's root directory. Websites contain files written in languages like PHP or HTML, which are served from a directory on the web server. For example, when you visit a website hosted through Apache, the web files are typically stored in ```/var/www/html```. If you access a website with the URL ```https://www.yourwebsite.com/index.php```, the ```index.php``` file is served from ```/var/www/html/index.php```. Similarly, images on the website might be stored in ```/var/www/html/images```, so an image file could be served from ```/var/www/html/images/image_1.png```.

In Linux, a pathname containing double dots (```..```) refers to the parent directory, which is one level up in the directory hierarchy. Therefore, ```/var/www/html/images/..``` and ```/var/www/html``` refer to the same directory, since ```..``` navigates up from ```images/``` to its parent directory, which is ```html```.

Web servers should restrict access to files within the web root directory to prevent unauthorized access to sensitive files. However, if a web server improperly validates user input, an attacker can craft a URL that includes directory traversal sequences to access files outside the web root. For example, an attacker might attempt to access ```/etc/passwd``` by requesting a URL like ```https://www.yourwebsite.com/index.php?file=../../../etc/passwd```.

This means that if a website has a vulnerability and allows traversal outside the web root, an attacker could access files like ```/etc/passwd```, which contains every user's account information on the server. Hackers can also view files in a user's home directory, such as ```/home/USERNAME_GOES_HERE/example_pic_1.jpg```. If Apache has access to these files, they can be displayed on the website.

This allows a pathname attack, where an attacker can "guess" the location of sensitive files and trick the server into reading and displaying their contents.

<strong>This lab will contain three topics, and you will learn the following:</strong>

1. Introduction to Flask
2. Path Canonicalization
3. Performing a Pathname Attack

In [1]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('"/home/USERNAME_GOES_HERE/resources/save.py pathname"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save(question, response, answer=""):
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()
    
    command = (f'"/home/.education/grader.py PA {question} {response}"' 
               if answer == "" 
               else f'"/home/.education/grader.py PA {question} {response} \\"{answer}\\""')
    
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

def load_notebook():
    with load_lock:
        result = subprocess.run('"/home/USERNAME_GOES_HERE/resources/load.py pathname"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# This variable is used to warn students if they haven't loaded a save yet.
def warn_student():
    if (os.path.exists("/home/USERNAME_GOES_HERE/saves/.pathname_warning")):
        os.remove("/home/USERNAME_GOES_HERE/saves/.pathname_warning")
        return True

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

### Step 0: Starting the Lab

Click the button to begin creating the experiment.

<strong>Note:</strong> If your buttons are not displaying, click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to render all widgets.

In [17]:
# Click the button below to start the experiment.
import time

def check_autosave():
    # Check if the student has an autosave.
    if os.path.exists("/home/USERNAME_GOES_HERE/saves/USERNAME_GOES_HERE_pathname.tar.gz"):
        # Create a hidden file that will serve as a "boolean" for later.
        subprocess.run("touch /home/USERNAME_GOES_HERE/saves/.pathname_warning", shell=True)

def startlab(button):
    # Defining the lab name.
    labname = "pathname"

    # Writing the information to an empty field below the button.
    with output0:
        # Fixes a strange error that happens only occasionally.
        os.chdir("/home/USERNAME_GOES_HERE")
        output0.clear_output()

        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = f"real.{labname}jup.USERNAME_GOES_HERE"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('"mrg list materializations" 2>/dev/null').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing activation for this lab already exists. </span><span>You might have run another \
            lab without stopping this one. Attaching the existing activation...</span>"))

            # Detaching it and re-attaching.
            subprocess.run('mrg xdc detach xdc.USERNAME_GOES_HERE', shell=True)
            subprocess.run(f'mrg xdc attach xdc.USERNAME_GOES_HERE real.{labname}jup.USERNAME_GOES_HERE', capture_output=True, text=True, shell=True)
            display(HTML("<span>Re-running the installation... </span><span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

            # Check if the student has an autosave.
            check_autosave()
            
            # Re-running the installation script.
            subprocess.run(f'bash /home/runlab {labname}jup', capture_output=True, text=True, shell=True)
            output0.clear_output()
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been re-installed. </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing activations are found.</span>"))

            # Second, start the lab.
            display(HTML(f"<span>Starting the {labname} lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            try:
                startexp = subprocess.run(f'bash /home/startexp {labname}jup', capture_output=True, text=True, shell=True)
            except Exception as e:
                output0.clear_output()
                display(HTML("<span style='color: red;'>There was an error starting your experiment. Make sure your password is written inside of ~/pass.txt, and try again.</span>"))
                return
            
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout, flush=True)

            # Another lab is already attached to the XDC.
            if ("XDC already attached" in startexp.stdout):
                existingLab = re.search(r"real.(.*).USERNAME_GOES_HERE", startexp.stdout).group(1)

                # Shouldn't happen.
                if labname == existingLab:
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML(f"<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the {existingLab} experiment.</span>"))
                    subprocess.run('mrg xdc detach xdc.USERNAME_GOES_HERE', shell=True)
                    display(HTML("<span>Attaching the current lab.</span>"))
                    subprocess.run(f'mrg xdc attach xdc {materialPattern}', shell=True)
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))

            # Gives the notebook a couple seconds so that it will recognize the node(s).
            time.sleep(2)

            # Move the resources over.
            runlab = subprocess.run(
                f'bash /home/runlab {labname}jup',
                capture_output=True, text=True, shell=True
            )

            # Check if the student has an autosave.
            check_autosave()

            # Complete. Inform the student.
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))


# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

<hr>

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/home/USERNAME_GOES_HERE/saves/USERNAME_GOES_HERE_pathname.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 1):
                output0_2.clear_output()
                display(HTML("<span style='color: green;'>Your lab has been successfully loaded. Please click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to reflect your changes.</span>"))
            elif (result.returncode == 2):
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>The buffer node is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

## <strong>Topic 1: Introduction to Flask</strong>

Many options exist for developing a web server, making it difficult to find a "best" language and framework for this lab. Other notebooks that are developed for this course mainly use PHP, which is easy to understand and is great with teaching security practices. However, to provide more experience with programming languages and frameworks, this notebook is going to introduce <strong>Flask</strong> for Python.

<strong>Flask</strong> is a web framework in Python. Note that Flask’s built-in server is <a href="https://stackoverflow.com/questions/63201661/why-should-i-switch-to-a-flask-production-deployment-instead-of-a-development-se">not suitable for production as it doesn’t scale well.</a> Flask is perfectly suitable for this notebook's needs: It's lightweight, it handles a small amount of requests, and it's designed to be quick and easy to start web applications. Flask is often more used for prototypes before being rolled out for production. If you wish to apply skills that you're learning from this lab, consider using Gunicorn (Green Unicorn), which can be used in conjunction with Flask. In fact, <a href="https://flask.palletsprojects.com/en/3.0.x/deploying/gunicorn/">Gunicorn is part of Flask's documentation</a>. If you're interested with learning the differences between the two, check out <a href="https://stackshare.io/stackups/flask-vs-gunicorn">this article</a>. 

When rolling out a large application that's developed in Flask, Gunicorn saves a lot of time and work with transitioning it from a Flask prototype to a large-scale web application. This is a basic summary of how Flask applications are transitioned into a large-scale app:
- Install Gunicorn
- Run a Gunicorn command that sets the amount of processes, IP, and the name of your Flask application.
- Create a ```systemd``` service file that makes the program run continuously in the background.

<u>This notebook is going to provide you the basic knowledge of Flask, but will not go in-depth with how to create a large, in-depth web application.</u> This will provide you some background before looking at a basic web application that's vulnerable to pathname attacks.

<strong>Port forwarding is required for this lab.</strong> If you have not set up port forwarding yet, navigate to <a href="port-forwarding-setup.ipynb">this notebook</a>, and follow the steps so that you may access the web server for this lab.

### Step 1: Configuring Port Forwarding

This step is not graded, but it is <strong>incredibly important</strong> for you to follow in order to access this lab. <strong>Accessing this lab is different from the other labs in these notebooks.</strong>

<u>Here's a quick start to accessing the lab:</u>
- After you have ran the setup from <a href="port-forwarding-setup.ipynb">the port forwarding process</a>, type the following command into your <u>own device's</u> terminal: ```ssh -L 5000:127.0.0.1:5001 USERNAME_GOES_HERE-xdc-USERNAME_GOES_HERE```.
- <u>From your local terminal OR your XDC</u>, log into your ```pathname``` node by typing the following command: ```ssh -L 5001:127.0.0.1:5000 pathname```.

<hr>

#### <strong>Optional reading below</strong> - What is this doing?

Flask automatically uses port 5000 on your ```pathname``` node. Additionally, it uses the IP address ```127.0.0.1```, which is the localhost IP address for ```pathname```. Unfortunately, this makes the port forwarding process complex for the lab.

In order to access this lab, using ```ssh pathname``` <em>will</em> work, but you will need to forward from port 5000 to 5001 either on your local machine, or on your XDC. You will need to introduce a "jump". Take look at this command:

```ssh -L 5001:127.0.0.1:5000 pathname```

This command will sign you into the node, but the ```-L``` flag means "local", since you are going to be opening a local port. After this flag, ```5001:127.0.0.1:5000``` means that you're going to take the resources hosted on ```127.0.0.1:5000```, then "wire" it to your local machine on port ```5001```. Which, in this case, is ```USERNAME_GOES_HERE@xdc```. This makes all resources that are hosted on port ```5000``` accessible on port ```5001```. You may test this by opening another terminal, then type ```curl localhost 5001```, which will print the output of your webpage as HTML.

Next, when using this command:

```ssh -L 5000:127.0.0.1:5001 USERNAME_GOES_HERE-xdc-USERNAME_GOES_HERE```

You are doing the same process as above. Now, the machine that you will access is ```USERNAME_GOES_HERE-xdc-USERNAME_GOES_HERE```, which is your XDC that you're doing this notebook and lab within. Similarly, you are taking the materials hosted on ```127.0.0.1:5001```, then you're hosting it on your local machine (your device) on port ```5000```.

Here's a breakdown of how these "jumps" work:

<figure><center><img src="resources/pathname/port-forwarding.png" style="width: 90%; height: 90%;" /></center></figure>

### Step 2: Basic Flask Example

Upon starting this lab, all dependencies have already been installed for you. For this step, you're going to learn how web pages are hosted from the Flask framework. This basic example that you are going to follow for Step 2 is from <a href="https://flask.palletsprojects.com/en/3.0.x/quickstart/">Flask's documentation</a>. It's not required to read this, but it will provide some additional information, such as entering debug mode or sanitizing user input (in case you're familiar with XSS).

Navigate into your ```pathname``` node, and create a file named ```step_2.py``` inside of your home directory. Copy this code into the file, then save. <strong>You are strongly encouraged to read the breakdown of this function.<strong>

```
from flask import Flask, render_template

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

if __name__ == "__main__":
    app.run()
```

<u>The code breakdown:</u>
- Import the Flask library.
- Creates a Flask class, then assigns it to the variable named ```app```. If you're unfamiliar with classes in Python, the ```__name__``` parameter is the constructor for Flask. This sets paths for static files and templates. Since everyone's installation for Flask contains different path names, this will let Flask determine where <em>your</em> files and templates are located.
- ```@app.route("/")``` is called a <strong>decorator</strong> in Python. Decorators server as "wrappers" around functions. Whenever a function is called with a decorator, it will call the decorator function first, then it will call the function that it "wraps" around.
  - In the context of Flask, the ```@app.route("/")``` function will tell Flask what URL will trigger the specified function.
- The ```def hello_world()``` function is what gets called when the decorator's webpage is accessed.

After you have created this file, stay in your home directory and run ```python3 -m flask --app step_2 run```. This will start your server. After running your SSH command to port forward, navigate to ```localhost:port```, where ```port``` was the number that you chose in your SSH command. If you need help with port forwarding, contact your professor or TA.

After your server is running, click the button below to check your work. The response from your server will be checked, which should return ```<p>Hello, World!</p>```.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> If you have correctly ran the commands above, you may access the server by navigating to ```http://localhost:5000``` on your computer.

In [4]:
# Click the button below to check your work.
step2Complete = False

# Function to check the permissions.
def step_2():
    # Important variables that must be accessed outside of this function.
    global step2Complete, result

    with output2:
        output2.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname /home/.checker/section_1.py 2', shell=True, capture_output=True, text=True)

    if (result.returncode == 0):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True

    elif (result.returncode == 1):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>Your server did not print \"Hello, World!\" properly. Make sure your file is exactly what what provided to you.</span>"))
            step2Complete = False

    elif (result.returncode == 2):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>step_2.py was not found in your home directory.</span>"))
            step2Complete = False

    elif (result.returncode == 3):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>Something went wrong with your lab's setup. Please contact your TA/instructor.</span>"))
            step2Complete = False
    
def check_step_2(b):
    if (warn_student()):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_2()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("2", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Server")

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(button, output2)

Button(description='Check Server', style=ButtonStyle())

Output()

### Step 3: Loading HTML with Flask

Upon completing Step 2, an HTML file was generated for you in your home directory named ```step_3.html```.

Additionally, a file was created for you called ```step_3.py```. This is just a copy of your ```step_2.py``` file.

Instead of returning ```<p>Hello, World!</p>```, you are going to render an entire HTML file. This introduces a <strong>template</strong>. Templates are HTML files that can be dynamically changed using Flask. Templates <u>must</u> be stored inside of a directory called ```templates/``` in your Flask directory. This notebook will not teach you how to write HTML or Flask templates.

Using your intuition, return the HTML file by using the ```render_template(source)``` command, which accepts an HTML file for its input.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> The ```templates/``` directory does not have to be included in ```render_template()```. When using this function, it will automatically search inside of the ```templates/``` directory for the corresponding file.

In [5]:
# Click the button below to check your work.
step3Complete = False

# Function to check the permissions.
def step_3():
    # Important variables that must be accessed outside of this function.
    global step3Complete, result

    with output3:
        output3.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname /home/.checker/section_1.py 3', shell=True, capture_output=True, text=True)

    if (result.returncode == 0):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step3Complete = True

    elif (result.returncode == 1):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Your server did not display step_3.html. Make sure that you are rendering an HTML file.</span>"))
            step3Complete = False

    elif (result.returncode == 2):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>step_3.py was not found in your home directory.</span>"))
            step3Complete = False

    elif (result.returncode == 3):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Something went wrong with your lab's setup. Please contact your TA/instructor.</span>"))
            step3Complete = False
    
def check_step_3(b):
    if (warn_student()):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_3()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("3", result.returncode)

# Creating the button.
button = widgets.Button(description="Check for Redirect")

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(button, output3)

Button(description='Check for Redirect', style=ButtonStyle())

Output()

### Step 4: Routing and Redirections

Upon completing Step 3, <strong> a developer made a new directory</strong> called ```/lab```. Inside of this directory, some files have already been made. Your focus is on the ```memo.py``` file.

<strong>Note:</strong> ```/lab``` has a leading slash in front of it, which means it's NOT in your home directory. You will find this directory inside of ```/```.

<u>For the rest of this section</u>, you are going to be implementing parts of a memo website, which will be used towards the end of the lab. A "developer" will be implementing the heavy work for you.

<strong>Routing</strong> in Flask is finding a specific URL, then making a function handle the logic for that URL. You may read more about routing <a href="https://www.geeksforgeeks.org/flask-app-routing/">here</a>.

For this step, use the ```redirect()``` command to redirect the user to `/` after <u>adding or deleting</u> a memo. The line of code that you need to change is provided to you in the file. For better practice, you're required to use ```url_for()```. You may read more about this function from the documentation <a href="https://flask.palletsprojects.com/en/3.0.x/api/#flask.url_for">here</a>. 

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span> 
- Note that ```url_for()``` takes one parameter, which is the name of a function in your Python file. Using ```url_for()``` is good practice, because if you change the location that your URL route points to, it will always refer to the updated URL, because it points to the function. If you struggle to understand this, think of it this way: ```url_for()``` is the "URL for your function". If ```def foo():``` has the route ```@app.route("/home/foo/foo.html")```, you can change the URL inside of this route, and the redirect will not need to be changed.
- Using ```redirect()``` should be fairly straightforward to implement. However, your notebook will be using this command to make sure that your redirect is working: ```curl POST -d "memo=test" -v http://127.0.0.1:5000/add_memo```. Feel free to try using this command in a second terminal, while your first terminal is hosting the server.
  - The answer that you use should be the same for ```add_memo()``` and ```delete_memo()```. The ```curl``` command above won't work for ```delete_memo()``` because there are currently no memos on the website yet.

In [6]:
# Click the button below to check your work.
step4Complete = False

# Function to check the permissions.
def step_4():
    # Important variables that must be accessed outside of this function.
    global step4Complete, result

    with output4:
        output4.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname /home/.checker/section_1.py 4', shell=True, capture_output=True, text=True)

    if (result.returncode == 0):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step4Complete = True

    elif (result.returncode == 1):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>Either add_memo() and/or delete_memo() did not have a correct redirect. Try again.</span>"))
            step4Complete = False

    elif (result.returncode == 2):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>The /lab/memo.py file was not found. Please contact your professor or TA for assistance.</span>"))
            step4Complete = False

    elif (result.returncode == 3):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>Something went wrong with your lab's setup. Please contact your TA/instructor.</span>"))
            step4Complete = False
    
def check_step_4(b):
    if (warn_student()):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_4()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("4", result.returncode)

# Creating the button.
button = widgets.Button(description="Check for Redirect")

# Creating an output area.
output4 = widgets.Output()

# Run the command on click.
button.on_click(check_step_4)

# Display the output.
display(button, output4)

Button(description='Check for Redirect', style=ButtonStyle())

Output()

### Step 5: Reading Input Through Flask

As you finished implementing redirections, the developer has updated the ```memo.py``` file to implement add/remove functionality for the memos. Here is how the functionality will work:
- ```add_memo()```
  - Take the content of the memo by using ```request.form.get```. This takes any ```<form>``` HTML attribute with the name ```memo```, then takes its contents.
  - If content exists, call ```get_next_memo_id()```. This is a separate function which gets all of the existing IDs, finds the highest value out of all of them, then adds one to it and returns it.
  - Create a filename for the new memo.
  - Write the file into the ```templates/memo/``` directory.
- ```view_memo()```
  - Take the ID of the memo and concatenate it with the ```memos/``` path.
  - Take the file path, then read the memo.
  - If the memo ID isn't numeric, then assign it to -1.
  - Print the memo onto the screen.
- ```load_memos()```
  - Create an empty dictionary called ```memos```
    - In Python, dictionaries are a set of key/value pairs, where the key is the ID of the memo, and the value is the content of it.
  - For each file in the memo directory, if it ends with ```.txt```, it's readable. Therefore, add the memo to the dictionary with the ID as the key, and the contents as the value.

The developer has decided to start implementing the HTML for the website. Meanwhile, they asked you to implement the ```delete_memo()``` function.

Note the parameters that are seen inside of the route calls. This is a breakdown of the parameter types that can be fed into the function as parameters:

| Type         | Description                                |
|--------------|--------------------------------------------|
| ```string``` | (default) accepts any text without a slash |
| ```int```    | accepts positive integers                  |
| ```float```  | accepts positive floating point values     |
| ```path```   | like ```string``` but also accepts slashes |
| ```uuid```   | accepts UUID strings                       |

<center><a href="https://flask.palletsprojects.com/en/3.0.x/quickstart/#variable-rules">Table Source</a></center>
<span>When parameters are passed from a route, they are available in the function that it wraps around.<br><br></span>

<strong>Your task:</strong> Update ```delete_memo``` so that it takes the ```<int:memo_id>``` variable, and deletes the memo with that ID. 

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>
- The other functions that the developer has implemented may help you. Specifically, seeing how the memo's paths are constructed would be a good start.
- Additionally, removing files can be done in Python using ```os.remove()```.
- When you access a memo, a "delete" button is already implemented, which you can use to test your work.

In [7]:
# Click the button below to check your work.
step5Complete = False

# Function to check the permissions.
def step_5():
    # Important variables that must be accessed outside of this function.
    global step5Complete, result

    with output5:
        output5.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname /home/.checker/section_1.py 5', shell=True, capture_output=True, text=True)

    if (result.returncode == 0):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step5Complete = True

    elif (result.returncode == 1):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Your delete_memo() function did not work. Try again.</span>"))
            step5Complete = False

    elif (result.returncode == 2):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Either /lab/memo.py file was not found, or the checker script is having an issue. Please contact your professor or TA for assistance.</span>"))
            step5Complete = False

    elif (result.returncode == 3):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>Something went wrong with your lab's setup. Please contact your TA/instructor.</span>"))
            step5Complete = False
    
def check_step_5(b):
    if (warn_student()):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_5()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("5", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Functionality")

# Creating an output area.
output5 = widgets.Output()

# Run the command on click.
button.on_click(check_step_5)

# Display the output.
display(button, output5)

Button(description='Check Functionality', style=ButtonStyle())

Output()

## <strong>Topic 2: Path Canonicalization</strong>

In the previous topic, you produced part of a memo website, which allows you to write memos, view them later, then delete them. Step 1 allowed you to view this website from your own browser.

Before looking for potential vulnerabilities with this memo website, you are going to be exploring how pathname attacks can be used to break websites. Once you have experimented with how pathnames work, you are going to find a vulnerability within the website that the other developer overlooked.

After you looked at how pathnames work, you are going to look at certain sanitizations, discover how to break the sanitizations, then look into the idea of canonicalizing a path. You will be experimenting with what happens when you call ```blog.php``` with a pathname attack. This PHP file assumes that it will take and format a text file. <strong>Other dangers can happen with pathname attacks.</strong> Suppose a PHP file deletes a blog or sends a file to somebody. Using pathname attacks, you can direct any file  to these PHP files and make the PHP files manipulate them.

### Step 6: Pathname Basics

After completing Topic 1, you have likely used the ```cd``` command to traverse directories. You should be familiar with the fact that ```..``` refers to the parent directory relative to your current location.

Suppose a website has the following URL:

<center style="color: blue; font-family:monospace; margin-bottom: 15px;">https://www.mywebsite.com/blog.php?file=posts/4183</center>

Upon inspection, this website contains a blog page that reads a file named `4183` on the server.

Now, suppose you know the location of this file on the server. The file is located in `/var/www/html/posts`, and the website itself is located in `/var/www/html`. Whenever the server needs to read a file, send an email, register a user, or update a database, all of its operations are conducted within this directory.

```blog.php``` reads files from ```/var/www/html/posts/XXX```, where ```XXX``` is the name of the file it reads. Using an injection attack, find a way to read the file named ```/etc/passwd``` by replacing the file name with parent directories (```..```) to traverse upward in the file hierarchy.

The template of the URL has already been provided. Simply type your response into the field below, then click the button to check your work.

<span style="color: orange"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> If you attempt to navigate back into the ```posts/``` directory (for example: ```posts/../posts/..```), your answer will not work. You may either start your answer with ```posts/``` or with the parent directory. <strong>You are required to use ```..``` in your attack.</strong>


In [8]:
# Click the button below to check your work.
step6Complete = False

# Function to save the short answer.
def step_6():
    # Important variables that must be accessed outside of this function.
    global step6Complete, result, user_input_quoted

    # Loading, in case saving is slow.
    with output6:
        output6.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    if (userInput6.value == ""):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>You did not type a response.</span>"))
            step6Complete = False

    else:
        user_input_quoted = (userInput6.value).strip()

        # Construct the SSH command for testing the student's response.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "/home/.checker/section_2.py 6 '{user_input_quoted}'" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "echo '{user_input_quoted}' > /home/.checker/responses/step_6_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: green;'>Your response is correct.</span>"))
                step6Complete = True
    
        elif (result.returncode == 1):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>Your response is incorrect.</span>"))
                step6Complete = False

def check_step_6(b):
    if (warn_student()):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_6()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("6", result.returncode, userInput6.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading6 = widgets.Output()
display(loading6)
with loading6:
    loading6.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Custom CSS for styling
custom_css = """
<style>
.custom-textarea .widget-label {
    white-space: normal !important;
    max-width: 600px !important;
    font-family: monospace;
    color: blue;
}
.custom-textarea .widget-text {
    width: 600px !important;
    font-family: monospace;
    color: blue;
}
</style>
"""

# Inject custom CSS
display(HTML(custom_css))

# Creating a text area with custom styling
userInput6 = widgets.Text(
    placeholder='Your Payload Here',
    description='https://www.mywebsite.com/blog.php?file=',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'}
)

# Apply custom class to the text input widget
userInput6.add_class('custom-textarea')

# Checking if the step has been answered
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "cat /home/.checker/responses/step_6_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput6.value = result.stdout

# Creating the button
button = widgets.Button(description="Check Path")

# After the student's response was loaded, clear the output.
loading6.clear_output()

# Creating an output area
output6 = widgets.Output()

# Attach the function to button click event
button.on_click(check_step_6)

# Display the widgets
display(userInput6, button, output6)

Output()

Text(value='', description='https://www.mywebsite.com/blog.php?file=', layout=Layout(width='600px'), placehold…

Button(description='Check Path', style=ButtonStyle())

Output()

### Step 7: Sanitization (Part 1)

From the previous step, you can clearly tell that this is a security risk. The ```/etc/passwd``` file leaks information about the users on the server. 

<strong>Note</strong>: This doesn't leak information about the users on the website. Rather, it leaks information about users from the computer that the website is hosted from.

Each line of the ```/etc/passwd``` file leaks the following information, <a href="https://www.ibm.com/docs/en/aix/7.1?topic=passwords-using-etcpasswd-file">from IBM</a>:

- User name
- Encrypted password
- User ID number (UID)
- User's group ID number (GID)
- Full name of the user (GECOS)
- User home directory
- Login shell

We cannot have this information viewable from the website. However, we cannot modify where this file exists, nor its permissions. Our website needs to sanitize the pathname.

<strong>Here is the sanitization function</strong>: Suppose that ```blog.php``` can detect if a pathname attack is occuring. If the website detects that you're navigating to the `/` directory, it will prepend the directory with this path name: ```/var/www/html/posts/```

Your task is to break this sanitization function. You are still supposed to use ```..``` in your payload.

<span style="color: orange"><strong><img src="resources/alert.png" style="width: 12px"> Notice:</strong></span> ```posts/``` is prepended to the filename. If you start your payload with ```posts/```, it will not work because ```/var/www/html/posts/posts/``` is an invalid directory!

In [9]:
# Click the button below to check your work.
step7Complete = False

# Function to save the short answer.
def step_7():
    # Important variables that must be accessed outside of this function.
    global step7Complete, result

    # Loading, in case saving is slow.
    with output7:
        output7.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    if (userInput7.value == ""):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>You did not type a response.</span>"))
            step7Complete = False

    else:
        user_input_quoted = (userInput7.value).strip()

        # Construct the SSH command for testing the student's response.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "/home/.checker/section_2.py 7 '{user_input_quoted}'" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "echo '{user_input_quoted}' > /home/.checker/responses/step_7_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: green;'>Your response is correct. (Notice that your payload from the previous question also works here?)</span>"))
                step7Complete = True
    
        elif (result.returncode == 1):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: red;'>Your response is incorrect.</span>"))
                step7Complete = False

def check_step_7(b):
    if (warn_student()):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_7()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("7", result.returncode, userInput7.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading7 = widgets.Output()
display(loading7)
with loading7:
    loading7.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Custom CSS for styling
custom_css = """
<style>
.custom-textarea .widget-label {
    white-space: normal !important;
    max-width: 600px !important;
    font-family: monospace;
    color: blue;
}
.custom-textarea .widget-text {
    width: 600px !important;
    font-family: monospace;
    color: blue;
}
</style>
"""

# Inject custom CSS
display(HTML(custom_css))

# Creating a text area with custom styling
userInput7 = widgets.Text(
    placeholder='Your Payload Here',
    description='https://www.mywebsite.com/blog.php?file=',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'}
)

# Apply custom class to the text input widget
userInput7.add_class('custom-textarea')

# Checking if the step has been answered
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "cat /home/.checker/responses/step_7_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput7.value = result.stdout

# Creating the button
button = widgets.Button(description="Check Path")

# After the student's response was loaded, clear the output.
loading7.clear_output()

# Creating an output area
output7 = widgets.Output()

# Attach the function to button click event
button.on_click(check_step_7)

# Display the widgets
display(userInput7, button, output7)

Output()

Text(value='', description='https://www.mywebsite.com/blog.php?file=', layout=Layout(width='600px'), placehold…

Button(description='Check Path', style=ButtonStyle())

Output()

### Step 8: Sanitization (Part 2)

After breaking the sanitization function from Step 7, you will need to find another way to fix this security issue.

<strong>Here is a new sanitization function</strong>: Prepending a path before a filename didn't work, so it will not be used anymore. If a ```..``` appears in the directory, remove it. Assume that the developer(s) for ```blog.php``` forgot to add a check, which allows absolute paths to be called from the command line.

Your task is to break this sanitization function.

In [10]:
# Click the button below to check your work.
step8Complete = False

# Function to save the short answer.
def step_8():
    # Important variables that must be accessed outside of this function.
    global step8Complete, result

    # Loading, in case saving is slow.
    with output8:
        output8.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    if (userInput8.value == ""):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>You did not type a response.</span>"))
            step8Complete = False

    else:
        user_input_quoted = (userInput8.value).strip()

        # Construct the SSH command for testing the student's response.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "/home/.checker/section_2.py 8 '{user_input_quoted}'" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "echo '{user_input_quoted}' > /home/.checker/responses/step_8_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output8.clear_output()
            with output8:
                display(HTML("<span style='color: green;'>Your response is correct.</span>"))
                step8Complete = True
    
        elif (result.returncode == 1):
            output8.clear_output()
            with output8:
                display(HTML("<span style='color: red;'>Your response is incorrect.</span>"))
                step8Complete = False

def check_step_8(b):
    if (warn_student()):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_8()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("8", result.returncode, userInput8.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading8 = widgets.Output()
display(loading8)
with loading8:
    loading8.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Custom CSS for styling
custom_css = """
<style>
.custom-textarea .widget-label {
    white-space: normal !important;
    max-width: 600px !important;
    font-family: monospace;
    color: blue;
}
.custom-textarea .widget-text {
    width: 600px !important;
    font-family: monospace;
    color: blue;
}
</style>
"""

# Inject custom CSS
display(HTML(custom_css))

# Creating a text area with custom styling
userInput8 = widgets.Text(
    placeholder='Your Payload Here',
    description='https://www.mywebsite.com/blog.php?file=',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'}
)

# Apply custom class to the text input widget
userInput8.add_class('custom-textarea')

# Checking if the step has been answered
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "cat /home/.checker/responses/step_8_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput8.value = result.stdout

# Creating the button
button = widgets.Button(description="Check Path")

# After the student's response was loaded, clear the output.
loading8.clear_output()

# Creating an output area
output8 = widgets.Output()

# Attach the function to button click event
button.on_click(check_step_8)

# Display the widgets
display(userInput8, button, output8)

Output()

Text(value='', description='https://www.mywebsite.com/blog.php?file=', layout=Layout(width='600px'), placehold…

Button(description='Check Path', style=ButtonStyle())

Output()

### Step 9: Sanitization (Part 3)

You should have discovered that Step 8 provided a sanitization function that didn't work.

<u>The developers of the website changed how their blogs are processed.</u> In order for the URL to open blog posts, ```posts/``` is no longer required in the ```file``` parameter. Here is an example of how a blog post can be read from the URL:

<center style="color: blue; font-family:monospace; margin-bottom: 15px;">https://www.mywebsite.com/blog.php?file=4183</center>

The file is still being read from ```/var/www/html/posts/```. The ```posts/``` portion of the URL was removed so that the next sanitization function will work.

<strong>Here is the new sanitization function</strong>: If a ```/``` appears in the pathname within ```file=XXXX```, remove it. The parent directory, ```..```, is acceptable, <strong>and is required for this step</strong>.

Your task is to break this sanitization function.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> There is still a way that unsafe characters can be processed within the URL. Take a look at <a href="https://www.degraeve.com/reference/urlencoding.php">this documentation</a> for a clue.

In [11]:
# Click the button below to check your work.
step9Complete = False

# Function to save the short answer.
def step_9():
    # Important variables that must be accessed outside of this function.
    global step9Complete, result

    # Loading, in case saving is slow.
    with output9:
        output9.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    if (userInput9.value == ""):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>You did not type a response.</span>"))
            step9Complete = False

    else:
        user_input_quoted = (userInput9.value).strip()

        # Construct the SSH command for testing the student's response.
        test_response = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "/home/.checker/section_2.py 9 '{user_input_quoted}'" """
        result = subprocess.run(test_response, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "echo '{user_input_quoted}' > /home/.checker/responses/step_9_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: green;'>Your response is correct.</span>"))
                step9Complete = True
    
        elif (result.returncode == 1):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: red;'>Your response is incorrect.</span>"))
                step9Complete = False

def check_step_9(b):
    if (warn_student()):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_9()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("9", result.returncode, userInput9.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading9 = widgets.Output()
display(loading9)
with loading9:
    loading9.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Custom CSS for styling
custom_css = """
<style>
.custom-textarea .widget-label {
    white-space: normal !important;
    max-width: 600px !important;
    font-family: monospace;
    color: blue;
}
.custom-textarea .widget-text {
    width: 600px !important;
    font-family: monospace;
    color: blue;
}
</style>
"""

# Inject custom CSS
display(HTML(custom_css))

# Creating a text area with custom styling
userInput9 = widgets.Text(
    placeholder='Your Payload Here',
    description='https://www.mywebsite.com/blog.php?file=',
    layout=widgets.Layout(width='600px'),
    style={'description_width': 'initial'}
)

# Apply custom class to the text input widget
userInput9.add_class('custom-textarea')

# Checking if the step has been answered
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "cat /home/.checker/responses/step_9_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput9.value = result.stdout

# Creating the button
button = widgets.Button(description="Check Path")

# After the student's response was loaded, clear the output.
loading9.clear_output()

# Creating an output area
output9 = widgets.Output()

# Attach the function to button click event
button.on_click(check_step_9)

# Display the widgets
display(userInput9, button, output9)

Output()

Text(value='', description='https://www.mywebsite.com/blog.php?file=', layout=Layout(width='600px'), placehold…

Button(description='Check Path', style=ButtonStyle())

Output()

### Step 10: Path Canonicalization

In order to properly detect traversal attacks, the idea of "canonicalizing" a path is introduced. Another term for this is called the "shortest absolute path". When a path is canonicalized, it returns the shortest possible branch from the top of the file hierarchy.

Here are some examples:

- ```canonical( /var/www/html/../../www/html )``` returns ```/var/www/html```. The final location that the URL returns is ```html```. Therefore, the shortest path to this directory from ```/``` is ```/var/www/html```.
- ```canonical( /home/USERNAME_GOES_HERE/../../var/www/html )``` returns ```/var/www/html```, for the same reason above.
- ```canonical( /home/USERNAME_GOES_HERE/../../etc/shadow/../udev/rules.d/../xattr.conf )``` returns ```/etc/xattr.conf``` because the final file within ```canonical()``` is ```xattr.conf```. The shortest distance from ```/``` to ```xattr.conf``` is ```/etc/xattr.conf```.

The ```canonical()``` function is pseudocode. In the prompt below, describe how the developers can fix their pathname attack by using this function.

In [12]:
# Click the button below to check your work.
step10Complete = False

# Function to save the short answer.
def step_10():
    # Important variables that must be accessed outside of this function.
    global step10Complete, result

    # Loading, in case saving is slow.
    with output10:
        output10.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    if (userInput10.value == ""):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'>You did not type a response.</span>"))
            step10Complete = False

    else:
        user_input = userInput10.value
        escaped_user_input = user_input.replace('"', '\\"').replace("'", "'\\''")

        # Send the student's response to the Python file.
        ssh_command = f'ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "echo \'{escaped_user_input}\' > /home/.checker/responses/step_10_answer.txt"'
        result = subprocess.run(ssh_command, shell=True)
        
        if (result.returncode == 0):
            output10.clear_output()
            with output10:
                display(HTML("<span style='color: red;'>There was an error saving your response.</span>"))
                step10Complete = False
    
        elif (result.returncode == 1):
            output10.clear_output()
            with output10:
                display(HTML("<span style='color: green;'>Your response was saved.</span>"))
                step10Complete = True

def check_step_10(b):
    if (warn_student()):
        output10.clear_output()
        with output10:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_10()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("10", result.returncode, userInput10.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading10 = widgets.Output()
display(loading10)
with loading10:
    loading10.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area. We will need to assign output of process to the value of this input.
userInput10 = widgets.Textarea(
    placeholder='Type your response here',
    description='Response:',
    layout=widgets.Layout(width='75%', height='150px', margin='10px')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "cat /home/.checker/responses/step_10_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Output creates a newline. Remove it.
userInput10.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading10.clear_output()

# Creating the feedback output area.
output10 = widgets.Output()

# Creating the button.
button = widgets.Button(description="Save Response")

# Run the command on click.
button.on_click(check_step_10)

# Display the output.
display(userInput10, button, output10)

Output()

Textarea(value='', description='Response:', layout=Layout(height='150px', margin='10px', width='75%'), placeho…

Button(description='Save Response', style=ButtonStyle())

Output()

## <strong>Topic 3: Performing a Pathname Attack</strong>

With your knowledge on pathname canonicalization, your final goal for this lab is to break your memo website with a pathname attack.

The reason why Topic 1 was so focused on how development in Flask works is so that it will be easier to understand how information is being passed from the URL to your Python file. Since you have worked a little bit in Flask, you should be able to analyze a critical part of how memos are viewed: The decorator uses a <strong>path</strong> as input.

### Step 11: Retrieve ```/etc/passwd``` From Your Website

Your website was written in Python, and not in PHP like the previous topic used for demonstrating pathname attacks. This means that <strong>your website does not accept URL parameters</strong>. This means you cannot access a memo by doing: ```localhost:5000/memo.php?id=1234```. Instead, your website is using a direct path to access memos on the website. You will need to think creatively with how a pathname attack can be sent to this Python decorator: ```@app.route('/memo/<path:memo_id>')```

Your website reads files like this: ```127.0.0.1:5000/memo/1```, which reads Memo 1 from your ```memos/``` directory. 

<strong>There's a slight issue</strong>: Your browser interprets ```../``` before it's sent to the server. This means that calling:

<center style="color: blue; font-family:monospace; margin-bottom: 15px;">http://127.0.0.1:5000/memo/../../../../../etc/passwd</center>

Returns you to this URL:

<center style="color: blue; font-family:monospace; margin-bottom: 15px;">http://127.0.0.1:5000/etc/passwd</center>

However, your website <u>returns files that are local to the web server</u>, meaning it's attempting to access ```/lab/etc/passwd```, which doesn't exist.

<strong>Your task</strong>: Discover a way to print ```/etc/passwd``` through the memo website that you worked on. Once you have a payload, copy/paste the <strong>payload</strong> that shows the content of ```/etc/passwd``` into the field below. <u>Do not include a URL.</u> Here is an example of the payload that you will need to copy:

<center style="color: blue; font-family:monospace; margin-bottom: 15px;">http://127.0.0.1:5000/memo/<u>your_payload_here</u></center>

In [13]:
# Click the button below to check your work.
step11Complete = False

# Function to check if the student's answer was correct.
def step_11():
    # Important variables that must be accessed outside of this function.
    global step11Complete, result

    # Loading, in case the check is slow.
    with output11:
        output11.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput11.value == ""):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step11Complete = False

    else:
        # Escape single quotes by replacing them with '\''.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput11.value.strip())
       
        # Construct the SSH command for testing the SQL query.
        test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "/home/.checker/section_3.py 11 '{escaped_user_input}'" """
        result = subprocess.run(test_command, shell=True, text=True, capture_output=True)
       
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "echo '{escaped_user_input}' > /home/.checker/responses/step_11_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
       
        if (result.returncode == 0):
            output11.clear_output()
            with output11:
                # Now, if the return code was 1, it returned the correct response. Print the HTML of the response:
                display(HTML("<span style='color: green;'>Your payload worked! Here is the output from your payload.</span>"))
                display(HTML(result.stdout))
                display(HTML("<hr>"))
                step11Complete = True
                
        elif (result.returncode == 1):
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: red;'>Your payload did not show the contents of /etc/passwd. The result is displayed below.</span>"))
                display(HTML(result.stdout))
                display(HTML("<hr>"))
                step11Complete = False

        elif (result.returncode == 2):
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: red;'>There was an error with the script. Please contact your professor or TA.</span>"))
                step11Complete = False

        elif (result.returncode == 3):
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: red;'>There was no \"etc\" or \"passwd\" detected in your payload. This is required to display the target file for the step.</span>"))
                step11Complete = False
       
def check_step_11(b):
    if (warn_student()):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_11()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("11", result.returncode, userInput11.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading11 = widgets.Output()
display(loading11)
with loading11:
    loading11.clear_output()
    display(HTML("<span>Loading your saved response... <img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput11 = widgets.Text(
    placeholder='Type your payload here',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname "cat /home/.checker/responses/step_11_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput11.value = result.stdout

# After the student's response was loaded, clear the output.
loading11.clear_output()

# Creating the button.
button = widgets.Button(description="Test Payload")

# Creating an output area.
output11 = widgets.Output()

# Run the command on click.
button.on_click(check_step_11)

# Display the output.
display(userInput11, button, output11)

Output()

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type your payload here')

Button(description='Test Payload', style=ButtonStyle())

Output()

### Step 12: Fixing the Vulnerability

You found the vulnerability on the website, and there are several different ways to fix it. For this activity, your goal for fixing the vulnerability is to focus on pathname canonicalization.

Not all languages have a built-in ```canonical()``` function. Fortunately, Python does, which is ```os.path.realpath()```. You may read about it from the documentation <a href="https://docs.python.org/3/library/os.path.html#os.path.realpath">here</a>.

In this step, <u>you cannot delete or change any of the existing code inside of ```view_memo()```</u>. Using ```os.path.realpath()```, check to see if the memo's path <u>starts</u> with the directory where the memos are stored. Otherwise, redirect the user back to ```index.html```.

Your solution is required to be typed within this code block in ```view_memo()```:

```
### Step 12 Solution START ###



### Step 12 Solution END ###
```

<strong>Writing code anywhere outside of this comment block will prevent you from passing this step. Do not delete the comments surrounding your answer.</strong> 

Python provides some easy-to-understand functions, which you may use for your solution. Here are some suggested functions. They will be required in your solution:
- ```os.path.realpath()```
- ```.startswith()```

In [14]:
# Click the button below to check your work.
step12Complete = False

# Function to check the permissions.
def step_12():
    # Important variables that must be accessed outside of this function.
    global step12Complete, result

    with output12:
        output12.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # This subprocess statement is a little different. Need to initiate environment variables at the same time when running the command.
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@pathname /home/.checker/section_3.py 12 NA', shell=True, capture_output=True, text=True)

    if (result.returncode == 0):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: green;'>Success! Your payload no longer displays /etc/passwd. It redirects back to the home page.</span>"))
            step12Complete = True

    elif (result.returncode == 1):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>Your payload appears to be mostly patched, but it does not redirect to the home page.</span>"))
            step12Complete = False

    elif (result.returncode == 3):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>A new user-defined function was detected within your memo.py file. Please remove it.</span>"))
            step12Complete = False

    elif (result.returncode == 4):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>There is code written outside of the designated solution block. You are only allowed to type your answer within the provided block.</span>"))
            step12Complete = False

    elif (result.returncode == 5):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>os.path.realpath wasn't found in your solution. This is required in your code block. It cannot be commented out.</span>"))
            step12Complete = False

    elif (result.returncode == 6):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>You do not have an answer for the previous step. You must complete the previous step before doing this step. You will need to revert your changes made to the file before the vulnerability works.</span>"))
            step12Complete = False

    elif (result.returncode == 7):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>A test memo was created, and accessing it redirected the script to the index page. Your patch should only redirect anything accessed outside of memos/.</span>"))
            step12Complete = False

    elif (result.returncode == 2):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>There was an error while checking this step. Please contact your professor or TA.</span>"))
            step12Complete = False
    
def check_step_12(b):
    if (warn_student()):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_12()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("12", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Fix")

# Creating an output area.
output12 = widgets.Output()

# Run the command on click.
button.on_click(check_step_12)

# Display the output.
display(button, output12)

Button(description='Check Fix', style=ButtonStyle())

Output()

## <strong>Grading</strong>

To check your overall grade, click on the button below.

In [15]:
# Click the button below to check your overall grade.
steps_to_check = [step_2, step_3, step_4, step_5, step_6, step_7, step_8, step_9, step_10, step_11, step_12]   

# Function to calculate grade after refreshing the cell
def calculate_grade(b):
    # To not auto-save at each step.
    global runAllSteps
    runAllSteps = True

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML("<span>Testing all steps. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Required for checking the boolean values.
    for func in steps_to_check:
        func()  # Call each function in order.

    # Assuming steps are updated above this cell in some way
    steps = [step2Complete, step3Complete, step4Complete, step5Complete, step6Complete, step7Complete, step8Complete, step9Complete, step10Complete, step11Complete, step12Complete]
    output = "<div style='color: orange;'>Step 1 is not graded.</div>"
    stepsCorrect = 1
    numOfSteps = len(steps)

    for i in range(numOfSteps):
        if steps[i]:
            stepsCorrect += 1
            output += "<div style='color: green;'>Step " + str(i + 2) + " is complete.</div>"
        else:
            output += "<div style='color: red;'>Step " + str(i + 2) + " is incomplete.</div>"

    output += "<div style='color: black;'>You have " + str(stepsCorrect) + " out of " + str(numOfSteps + 1) + " steps completed.</div>"

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML(output))

    # Allows the individual steps to be auto-graded again.
    runAllSteps = False

# Create a button to refresh the cell and another to calculate grade.
grade_button = widgets.Button(description="Calculate Grade")

# Link buttons to functions.
grade_button.on_click(calculate_grade)

# Output area.
gradeOutput = widgets.Output()

# Display the buttons and output.
display(grade_button, gradeOutput)

Button(description='Calculate Grade', style=ButtonStyle())

Output()

### Stopping the Lab

Once you are done with the lab, click on the "Stop Lab" button below. <strong>This will delete your materialization, which will delete all of the lab's resources.</strong> Your progress is saved automatically in ```saves/``` within the sidebar of your XDC. You may load this lab in the future by clicking "Load Lab" at the top.

In [16]:
# Click the button below to stop the experiment.
def stoplab(button):
    # Check to make sure that the student wants to confirm ending the lab.
    if (confirm.value == False):
        with stop_output:
            stop_output.clear_output()
            display(HTML("<newline><span style='color: red;'>Please confirm that you wish to end the lab.</span>"))

    else:
        # Defining the lab name.
        labname = "pathname"
    
        # Writing the information to an empty field below the button.
        with stop_output:
            stop_output.clear_output()
            
            display(HTML("<span>Stopping the " + labname + " lab. This will take a minute to process. Please wait.</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            stopexp = subprocess.run('"bash /share/stopexp ' + labname + 'jup"', capture_output=True, text=True, shell=True)
            stop_output.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(stopexp.stdout)
    
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been ended.</strong></span>"))

# Creating the button.
stopButton = widgets.Button(description="Stop Lab")

# Create a confirmation check.
confirm = widgets.Checkbox(
    value=False,
    description='Confirm',
    disabled=False,
    indent=False
)

# Creating an output area.
stop_output = widgets.Output()

# Run the command on click.
stopButton.on_click(stoplab)

# Display the output.
display(confirm, stopButton, stop_output)

Checkbox(value=False, description='Confirm', indent=False)

Button(description='Stop Lab', style=ButtonStyle())

Output()